In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
download = drive.CreateFile({'id': '16hA5xU-EmCSRAVEkK_xtYQv8FWyvHOl6'})
download.GetContentFile('TomatoData.zip')

In [ ]:
!unzip TomatoData.zip
!rm TomatoData.zip

In [ ]:
# Part 1 - Building the CNN

# Importing the keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

# Initialising the CNN or classifier
classifier = Sequential()

# Step 1 - Convolution, pooling!
classifier.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
classifier.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=2))
classifier.add(BatchNormalization())

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 4096, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 3, activation = 'sigmoid'))

# Compiling the CNN or classifier
classifier.compile(optimizer = Adam(lr=3e-06, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True), loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def plotHistory(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()    

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('TomatoData/train', target_size = (224, 224),batch_size = 32,class_mode = 'categorical')

validation_set = test_datagen.flow_from_directory('TomatoData/val', target_size = (224, 224), batch_size = 32, class_mode = 'categorical')

try:
  history = classifier.fit_generator(training_set, steps_per_epoch = 94, callbacks = [EarlyStopping('val_acc', mode='auto', patience=3)], epochs = 50, validation_data = validation_set, validation_steps = 37)
  plotHistory(history)
finally:
  classifier.save('model.h5')

In [ ]:
# upload weights file to google drive.
upload = drive.CreateFile({'title': 'model.h5'})
upload.SetContentFile('model.h5')
upload.Upload()
print('Uploaded file with ID {}'.format(upload.get('id')))